In [13]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.gconvGRU_simple import GConvGRUModel
from memory_capacity_utils import gen_lag_data, compute_memory_capacity_vectorized, get_mem_cap_from_model

In [14]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')

running on GPU


# Load the dataset

In [27]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=3, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learning rate")
    parser.add_argument('--memcap_coef', type=float, default=0.00005, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [28]:
args = get_args()

In [29]:
dataset = np.load('datasets/oasis_data.npy')
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
dataset = np.delete(dataset,88,axis=0)

In [30]:
print(dataset.shape)

torch.Size([113, 3, 35, 35])


# Set up the network architecture and train-validate

**Train-validate-functions**

In [33]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGRU_t/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_gconvGR

In [34]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [35]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [38]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_oasis.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
   
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    model = GConvGRUModel(device=device, input_weights=input_weights, input_scaling=1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        mae_mem_cap_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
            mem_cap_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1):
                pred, output_sig = model(data[t], X_train_res, y_train_res, X_test_res)
                    
                real = data[t + 1]
        
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

                # MAE between predicted graph's mem cap and actual graph's mem cap
                predicted_mem_cap = compute_memory_capacity_vectorized(output_sig, y_test_res)
                actual_mem_cap = torch.tensor(train_mem_cap_subjects[data_id, t + 1], requires_grad=True).to(device)
                mem_cap_loss += mael(predicted_mem_cap, actual_mem_cap)

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Calculate the total MAE between Mem Cap Loss for the current batch
            mem_cap_loss = mem_cap_loss / (args.num_timepoints - 1)
            
            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            mae_mem_cap_overall.append(mem_cap_loss.item())
            
            total_loss = mae_loss + args.memcap_coef * mem_cap_loss 
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        mae_mem_cap_overall = np.mean(np.array(mae_mem_cap_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}, MAE of Mem Caps Loss: {mae_mem_cap_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/3]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.05094839483499527, TP Loss: 3.404314124584198, MAE of Mem Caps Loss: 0.21535666954777496
[Validate] MAE Loss Across Timepoints: [0.0768711  0.06449318]
[Validate] TP Loss Across Timepoints: [11.10978597  2.19295807]
[Validate] MAE of Mem Caps Across Timepoints: [0.58827517 0.96114863]
Epoch [2/20]
[Train] MAE Loss: 0.04332901205246647, TP Loss: 2.7390817160407703, MAE of Mem Caps Loss: 0.18753290818542928
[Validate] MAE Loss Across Timepoints: [0.0653088  0.05354968]
[Validate] TP Loss Across Timepoints: [9.27441813 1.54202385]
[Validate] MAE of Mem Caps Across Timepoints: [0.54964615 0.87466238]
Epoch [3/20]
[Train] MAE Loss: 0.03854758872961005, TP Loss: 2.310478690514962, MAE of Mem Caps Loss: 0.19689558895469253
[Validate] MAE Loss Across Timepoints: [0.06133409 0.04943677]
[Validate] TP Loss Across Timepoints: [8.76549072 1.36646042]
[Validate] MAE of Mem Caps A

[Validate] MAE Loss Across Timepoints: [0.05872123 0.05481744]
[Validate] TP Loss Across Timepoints: [8.47408498 3.00945002]
[Validate] MAE of Mem Caps Across Timepoints: [0.50332834 0.64515168]
Epoch [6/20]
[Train] MAE Loss: 0.033454102495064336, TP Loss: 1.6567892198761305, MAE of Mem Caps Loss: 0.18964640375411482
[Validate] MAE Loss Across Timepoints: [0.05578137 0.05182543]
[Validate] TP Loss Across Timepoints: [7.98954315 2.645578  ]
[Validate] MAE of Mem Caps Across Timepoints: [0.41195098 0.69209119]
Epoch [7/20]
[Train] MAE Loss: 0.032488804496824744, TP Loss: 1.5879916677872339, MAE of Mem Caps Loss: 0.1603063301146405
[Validate] MAE Loss Across Timepoints: [0.05545097 0.0514492 ]
[Validate] TP Loss Across Timepoints: [8.03735504 2.61292826]
[Validate] MAE of Mem Caps Across Timepoints: [0.40032419 0.67066019]
Epoch [8/20]
[Train] MAE Loss: 0.031189000885933637, TP Loss: 1.487103994190693, MAE of Mem Caps Loss: 0.1523794432894539
[Validate] MAE Loss Across Timepoints: [0.0529

[Train] MAE Loss: 0.030435808065036932, TP Loss: 1.6540501333773137, MAE of Mem Caps Loss: 0.21131433407366745
[Validate] MAE Loss Across Timepoints: [0.04149675 0.04467349]
[Validate] TP Loss Across Timepoints: [1.65308249 1.77500248]
[Validate] MAE of Mem Caps Across Timepoints: [0.6059683  0.62696106]
Epoch [11/20]
[Train] MAE Loss: 0.030288539143900076, TP Loss: 1.628926435112953, MAE of Mem Caps Loss: 0.21889586569062613
[Validate] MAE Loss Across Timepoints: [0.04132172 0.04438901]
[Validate] TP Loss Across Timepoints: [1.59604812 1.70661378]
[Validate] MAE of Mem Caps Across Timepoints: [0.62007723 0.60197215]
Epoch [12/20]
[Train] MAE Loss: 0.03027976133550207, TP Loss: 1.6222622953355312, MAE of Mem Caps Loss: 0.21909706740823864
[Validate] MAE Loss Across Timepoints: [0.0413506  0.04440737]
[Validate] TP Loss Across Timepoints: [1.60766804 1.72674608]
[Validate] MAE of Mem Caps Across Timepoints: [0.58000034 0.60906532]
Epoch [13/20]
[Train] MAE Loss: 0.03008533806229631, TP 